# Data Preparation Notebook

Notebook to prepare data and extract features for the project.

## Install required packages

In [1]:
!pip install python-dotenv
!pip install pandas
!pip install openai==2.8.1

## Imports

Import any packages required for this project.

In [2]:
import ast
import os
import requests
import json
import time
import re
from datetime import datetime, timedelta

import pandas as pd
from IPython.display import display
from dotenv import load_dotenv
from openai import OpenAI

## Load env variables

Load environment variables.

In [3]:
load_dotenv()

True

## Variables

Variables required throughout the whole notebook

In [4]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

data_folder = "../../data"
raw_data_folder = f"{data_folder}/raw"
staging_data_folder = f"{data_folder}/staging"

local_news_articles_csv = f"{raw_data_folder}/local_news_articles.csv"
police_press_releases_csv = f"{raw_data_folder}/police_press_releases.csv"

## Explanation & Mini Preparation

High-level explanation of the process as follows:

1. We first extract certain features using regex from the original CSV files.
2. We use LLMs to extract other features from the original CSV files.
3. Combine everything together into one CSV file for manual auditing.
4. After we manually audit, we combine both of the audit CSV files together and deduplicate
5. On the deduplicated data, extract other features using rule-based processing and other dimensions (date, street, town).

### News Articles Preparation

In [5]:
local_news_articles_df = pd.read_csv(local_news_articles_csv)

print(f"Dataset shape: {local_news_articles_df.shape}")
print(f"\nColumn names: {local_news_articles_df.columns.tolist()}")
print(f"\nPrimary key range: {local_news_articles_df["article_id"].min()} to {local_news_articles_df["article_id"].max()}")

print("\nDataset Info:")
display(local_news_articles_df.info())

print("\nFirst few rows:")
display(local_news_articles_df.head())

Dataset shape: (321, 14)

Column names: ['article_id', 'url', 'source_name', 'source_url', 'title', 'subtitle', 'author_name', 'publish_date', 'content', 'top_image_url', 'top_image_caption', 'created_at', 'tags', 'categories']

Primary key range: 40 to 496772

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   article_id         321 non-null    int64 
 1   url                321 non-null    object
 2   source_name        321 non-null    object
 3   source_url         321 non-null    object
 4   title              321 non-null    object
 5   subtitle           313 non-null    object
 6   author_name        321 non-null    object
 7   publish_date       321 non-null    object
 8   content            321 non-null    object
 9   top_image_url      318 non-null    object
 10  top_image_caption  312 non-null    object
 11  cre

None


First few rows:


,article_id,url,source_name,source_url,title,subtitle,author_name,publish_date,content,top_image_url,top_image_caption,created_at,tags,categories
0,4208,https://timesofmalta.com/article/driver-stuck-...,Times of Malta,https://timesofmalta.com,Driver stuck in traffic says speeding LESA car...,‘I was shocked at that moment but more so frus...,Emma Borg,2024-12-07,A motorist claims his car mirror was shattered...,https://cdn-attachments.timesofmalta.com/706da...,The broken car mirror. Photo: Frank Xerri De Caro,2025-07-03 15:14:21.554132+00,"{Accident,Lesa,National}",{}
1,4167,https://timesofmalta.com/article/pn-slams-gove...,Times of Malta,https://timesofmalta.com,PN slams government for diverting EU bus funds...,"'By encouraging the use of private cars, the g...",Times of Malta,2024-12-09,The PN on Monday slammed the government for di...,https://cdn-attachments.timesofmalta.com/d9afe...,"PN spokespeople Ryan Callus, Mark Anthony Samm...",2025-07-03 15:14:10.643172+00,"{""Climate Change"",Environment,""European Union""...",{}
2,4093,https://timesofmalta.com/article/motorcyclist-...,Times of Malta,https://timesofmalta.com,Motorcyclist seriously hurt in St Paul's Bay b...,Residents complained several times about inade...,Times of Malta,2024-12-11,A motorcyclist was rushed to hospital in a cri...,https://cdn-attachments.timesofmalta.com/633f6...,Photo: Malta Police Force,2025-07-03 15:13:50.605708+00,"{Accident,National,""St Paul’S Bay"",Traffic}",{}
3,4110,https://timesofmalta.com/article/skip-involved...,Times of Malta,https://timesofmalta.com,Skip involved in horror St Paul’s Bay bypass c...,Motorcyclist hurt in crash on Wednesday evenin...,Emma Borg,2024-12-12,A private contractor who placed a skip on St P...,https://cdn-attachments.timesofmalta.com/fc23e...,A 54-year-old man was seriously injured when h...,2025-07-03 15:13:54.812813+00,"{Accident,National,""St Paul’S Bay""}",{}
4,4066,https://timesofmalta.com/article/two-people-in...,Times of Malta,https://timesofmalta.com,"Two people, including teenage girl, critically...",Incidents in Mellieħa and Gudja on Friday even...,Times of Malta,2024-12-14,A 29-year-old man and 17-year-old girl were cr...,https://cdn-attachments.timesofmalta.com/f1761...,The Ford Fiesta involved in the Gudja collisio...,2025-07-03 15:13:43.83839+00,"{Accident,Gudja,Mellieħa,National,Traffic}",{}


### Police Press Releases Preparation

In [6]:
police_press_releases_df = pd.read_csv(police_press_releases_csv)
police_press_releases_df.insert(0, 'release_id', range(1, len(police_press_releases_df) + 1))

print(f"Dataset shape: {police_press_releases_df.shape}")
print(f"\nColumn names: {police_press_releases_df.columns.tolist()}")
print(f"\nPrimary key range: {police_press_releases_df["release_id"].min()} to {police_press_releases_df["release_id"].max()}")

print("\nDataset Info:")
display(police_press_releases_df.info())

print("\nFirst few rows:")
display(police_press_releases_df.head())

Dataset shape: (111, 5)

Column names: ['release_id', 'title', 'date_published', 'date_modified', 'content']

Primary key range: 1 to 111

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   release_id      111 non-null    int64 
 1   title           111 non-null    object
 2   date_published  111 non-null    object
 3   date_modified   111 non-null    object
 4   content         111 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.5+ KB


None


First few rows:


,release_id,title,date_published,date_modified,content
0,1,Collision between a car and a motorbike in Żur...,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were info..."
1,2,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were ..."
2,3,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were info..."
3,4,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were ..."
4,5,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the ev..."


## 1. Regex Extraction

Extract datetime from both police press releases and local news articles using regex.

### Variables

In [7]:
regex_extract_data_folder = f"{staging_data_folder}/regex_extract"

### News Articles

Extract accident datetime from news articles.

#### Methods

In [8]:
def classify_article_road_accident(row):
    """Classify if article is about a road accident."""
    text = (str(row['title']) + ' ' + str(row['content'])).lower()
    tags = str(row['tags']).lower()
    
    # exclude policy articles (government, budget, legislation, etc.)
    policy_keywords = ['government', 'minister', 'policy', 'budget', 'funds', 'legislation', 'parliament', 'proposal', 'grant', 'incentive', 'subsidy']
    if sum(1 for k in policy_keywords if k in text) >= 3:
        return 0
    
    # exclude non-accident traffic incidents
    non_accident_keywords = ['speeding', 'speed gun', 'caught doing', 'clocked at', 'pothole', 'flat tyre', 'flat tire', 'road damage', 'traffic violation', 'employer']
    if any(k in text for k in non_accident_keywords):
        return 0
    
    person_vehicle_terms = ['motorcyclist', 'cyclist', 'pedestrian']
    accident_keywords = ['crash', 'collision', 'injured', 'grievously injured', 'seriously injured', 'hit by', 'overturned', 'lost control', 'hit-and-run', 'run over']
    vehicles = ['car', 'bus', 'truck', 'van', 'motorcycle', 'bike', 'bicycle', 'scooter', 'vehicle']
    
    has_person_vehicle = any(k in text for k in person_vehicle_terms)
    has_accident_keyword = any(k in text for k in accident_keywords)
    has_vehicle = any(v in text for v in vehicles)
    
    if 'accident' in tags and (has_vehicle or has_accident_keyword or has_person_vehicle):
        return 1
    if has_accident_keyword and (has_vehicle or has_person_vehicle):
        return 1
    if has_person_vehicle and (has_accident_keyword or 'accident' in tags):
        return 1
    
    return 0

In [9]:
def parse_time_to_datetime(date_obj, time_str):
    if not time_str or pd.isna(date_obj):
        return None
    try:
        time_str = time_str.lower().strip().replace('.', ':')
        time_obj = datetime.strptime(time_str, '%I:%M%p' if ':' in time_str else '%I%p').time()
        date_only = date_obj.date() if isinstance(date_obj, pd.Timestamp) else (date_obj.date() if hasattr(date_obj, 'date') else date_obj)
        return datetime.combine(date_only, time_obj)
    except:
        return None

In [10]:
def extract_article_accident_datetime(row):
    text = str(row['content']).lower()
    title = str(row['title']).lower()
    try:
        pub_date = pd.to_datetime(row['publish_date'])
    except:
        pub_date = None
    
    # time patterns
    time_patterns = [
        r'at (?:around |about )?(\d{1,2}(?:\.\d{2})?(?:am|pm))',
        r'at (?:around |about )?(\d{1,2}:\d{2})\s*([ap]\.?m\.?)',
        r'(?:reported|occurred|happened|took place) at (?:around |about )?(\d{1,2}:\d{2})\s*([ap]\.?m\.?)',
        r'(?:reported|occurred|happened|took place) at (?:around |about )?(\d{1,2}(?:\.\d{2})?(?:am|pm))',
        r'at (?:around |about )?(\d{1,2}:\d{2})',  # Fallback without am/pm
    ]
    
    extracted_time = None
    for pattern in time_patterns:
        match = re.search(pattern, text)
        if match:
            if len(match.groups()) == 2:
                # pattern with separate time and am/pm groups
                time_part = match.group(1)
                ampm_part = match.group(2).replace('.', '').replace(' ', '')
                extracted_time = time_part + ampm_part
            else:
                extracted_time = match.group(1)
            break
    
    # day patterns
    day_patterns = {
        r'on monday': ('Monday', 0), 
        r'on tuesday': ('Tuesday', 1), 
        r'on wednesday': ('Wednesday', 2),
        r'on thursday': ('Thursday', 3), 
        r'on friday': ('Friday', 4), 
        r'on saturday': ('Saturday', 5),
        r'on sunday': ('Sunday', 6), 
        r'this (?:morning|afternoon|evening|night)': ('today', None),
        r'(?:yesterday|last night)': ('yesterday', None),
        r'last (?:monday|tuesday|wednesday|thursday|friday|saturday|sunday)': ('last_week', -7),
    }
    
    extracted_day = None
    accident_date = None
    
    # check both title and content for day references
    combined_text = title + ' ' + text
    
    for pattern, (day_value, weekday) in day_patterns.items():
        if re.search(pattern, combined_text):
            extracted_day = day_value
            if pub_date is not None:
                if day_value == 'today':
                    accident_date = pub_date
                elif day_value == 'yesterday':
                    accident_date = pub_date - timedelta(days=1)
                elif day_value == 'last_week':
                    # For "last Monday", etc - go back to that day in the previous week
                    target_day = pattern.split()[-1].rstrip(r')')
                    day_map = {'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 
                              'friday': 4, 'saturday': 5, 'sunday': 6}
                    if target_day in day_map:
                        target_weekday = day_map[target_day]
                        days_back = (pub_date.weekday() - target_weekday) % 7
                        if days_back == 0:
                            days_back = 7  # If same day, go back full week
                        accident_date = pub_date - timedelta(days=days_back)
                elif weekday is not None:
                    # Calculate days back, ensuring we go into the past
                    days_back = (pub_date.weekday() - weekday) % 7
                    if days_back == 0:
                        # If same weekday as publish date, assume it's today (0 days back)
                        days_back = 0
                    accident_date = pub_date - timedelta(days=days_back)
            break
    
    # if no specific day found, assume accident date = publish date
    if accident_date is None and pub_date is not None:
        accident_date = pub_date
    
    # combine date and time
    if extracted_time:
        accident_datetime = parse_time_to_datetime(accident_date, extracted_time)
    else:
        accident_datetime = pd.Timestamp(accident_date) if accident_date is not None else None
    
    # CRITICAL: Ensure accident_datetime is never after publish_date
    # compare dates only (not times) to avoid false positives on same-day accidents
    if accident_datetime is not None and pub_date is not None:
        accident_date_only = accident_datetime.date() if hasattr(accident_datetime, 'date') else accident_datetime
        pub_date_only = pub_date.date() if hasattr(pub_date, 'date') else pub_date
        
        if accident_date_only > pub_date_only:
            # If date is in the future, go back one week to the same weekday
            accident_datetime = accident_datetime - timedelta(days=7)
            accident_date = accident_datetime.date() if hasattr(accident_datetime, 'date') else accident_datetime
    
    return extracted_time, extracted_day, accident_date, accident_datetime

#### Main Logic

In [11]:
regex_extract_news_articles_csv = f"{regex_extract_data_folder}/news_articles.csv"

if os.path.isfile(regex_extract_news_articles_csv):
    print("Regex Extraction from news articles was already done")
else:
    # use regex to determine whether or not the article is a road accident
    regex_extract_news_articles_df = local_news_articles_df.copy()
    regex_extract_news_articles_df['is_road_accident'] = regex_extract_news_articles_df.apply(classify_article_road_accident, axis=1)

    # use regex to extract datetime
    (
        regex_extract_news_articles_df['accident_time'],
        regex_extract_news_articles_df['accident_day'],
        regex_extract_news_articles_df['accident_date'],
        regex_extract_news_articles_df['accident_datetime']
    ) = zip(*regex_extract_news_articles_df.apply(extract_article_accident_datetime, axis=1))

    regex_extract_news_articles_df['time_confidence'] = regex_extract_news_articles_df.apply(
        lambda row: 'High' if pd.notna(row['accident_time']) else ('Medium' if pd.notna(row['accident_day']) else 'Low'), axis=1
    )

    regex_extract_news_articles_df['accident_hour'] = regex_extract_news_articles_df['accident_datetime'].apply(lambda dt: dt.hour if pd.notna(dt) else None)

    regex_extract_news_articles_df['accident_is_weekend'] = regex_extract_news_articles_df['accident_datetime'].apply(
        lambda dt: 1 if (pd.notna(dt) and dt.weekday() >= 5) else (0 if pd.notna(dt) else None)
    )

    regex_extract_news_articles_df['publication_delay_hours'] = regex_extract_news_articles_df.apply(
        lambda row: max(0, (pd.to_datetime(row['publish_date']) - row['accident_datetime']).total_seconds() / 3600) if pd.notna(row['accident_datetime']) else None, axis=1
    )

    # show stats for accidents only
    accidents_df = regex_extract_news_articles_df[regex_extract_news_articles_df['is_road_accident'] == 1].copy()
    print(f"High: {(accidents_df['time_confidence'] == 'High').sum()} | Medium: {(accidents_df['time_confidence'] == 'Medium').sum()} | Low: {(accidents_df['time_confidence'] == 'Low').sum()}")
    print(f"\nTotal articles processed: {len(regex_extract_news_articles_df)}")
    print(f"\nAccidents identified: {len(accidents_df)} ({len(accidents_df)/len(regex_extract_news_articles_df)*100:.1f}%)")

    columns_to_save = [col for col in regex_extract_news_articles_df.columns if col not in ['accident_day', 'accident_hour']]
    regex_extract_news_articles_df[columns_to_save].to_csv(regex_extract_news_articles_csv, index=False)

    print(f"✓ Saved {len(regex_extract_news_articles_df)} articles to {regex_extract_news_articles_csv}")
    print(f"  - Road accidents: {(regex_extract_news_articles_df['is_road_accident'] == 1).sum()}")
    print(f"  - Other articles: {(regex_extract_news_articles_df['is_road_accident'] == 0).sum()}")
    print(f"  - Articles with datetime extracted: {regex_extract_news_articles_df['accident_datetime'].notna().sum()}")

regex_extract_news_articles_df = pd.read_csv(regex_extract_news_articles_csv)
display(regex_extract_news_articles_df)

High: 144 | Medium: 66 | Low: 25

Total articles processed: 321

Accidents identified: 235 (73.2%)
✓ Saved 321 articles to ../../data/staging/regex_extract/news_articles.csv
  - Road accidents: 235
  - Other articles: 86
  - Articles with datetime extracted: 320


,article_id,url,source_name,source_url,title,subtitle,author_name,publish_date,content,top_image_url,...,created_at,tags,categories,is_road_accident,accident_time,accident_date,accident_datetime,time_confidence,accident_is_weekend,publication_delay_hours
0,4208,https://timesofmalta.com/article/driver-stuck-...,Times of Malta,https://timesofmalta.com,Driver stuck in traffic says speeding LESA car...,‘I was shocked at that moment but more so frus...,Emma Borg,2024-12-07,A motorist claims his car mirror was shattered...,https://cdn-attachments.timesofmalta.com/706da...,...,2025-07-03 15:14:21.554132+00,"{Accident,Lesa,National}",{},0,NaN,2024-12-04,2024-12-04 00:00:00,Medium,0.0,72.0
1,4167,https://timesofmalta.com/article/pn-slams-gove...,Times of Malta,https://timesofmalta.com,PN slams government for diverting EU bus funds...,"'By encouraging the use of private cars, the g...",Times of Malta,2024-12-09,The PN on Monday slammed the government for di...,https://cdn-attachments.timesofmalta.com/d9afe...,...,2025-07-03 15:14:10.643172+00,"{""Climate Change"",Environment,""European Union""...",{},0,NaN,2024-12-09,2024-12-09 00:00:00,Medium,0.0,0.0
2,4093,https://timesofmalta.com/article/motorcyclist-...,Times of Malta,https://timesofmalta.com,Motorcyclist seriously hurt in St Paul's Bay b...,Residents complained several times about inade...,Times of Malta,2024-12-11,A motorcyclist was rushed to hospital in a cri...,https://cdn-attachments.timesofmalta.com/633f6...,...,2025-07-03 15:13:50.605708+00,"{Accident,National,""St Paul’S Bay"",Traffic}",{},1,5pm,2024-12-11,2024-12-11 17:00:00,High,0.0,0.0
3,4110,https://timesofmalta.com/article/skip-involved...,Times of Malta,https://timesofmalta.com,Skip involved in horror St Paul’s Bay bypass c...,Motorcyclist hurt in crash on Wednesday evenin...,Emma Borg,2024-12-12,A private contractor who placed a skip on St P...,https://cdn-attachments.timesofmalta.com/fc23e...,...,2025-07-03 15:13:54.812813+00,"{Accident,National,""St Paul’S Bay""}",{},1,1pm,2024-12-11,2024-12-11 13:00:00,High,0.0,11.0
4,4066,https://timesofmalta.com/article/two-people-in...,Times of Malta,https://timesofmalta.com,"Two people, including teenage girl, critically...",Incidents in Mellieħa and Gudja on Friday even...,Times of Malta,2024-12-14,A 29-year-old man and 17-year-old girl were cr...,https://cdn-attachments.timesofmalta.com/f1761...,...,2025-07-03 15:13:43.83839+00,"{Accident,Gudja,Mellieħa,National,Traffic}",{},1,5.30pm,2024-12-13,2024-12-13 17:30:00,High,0.0,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,496574,https://timesofmalta.com/article/watch-msida-f...,Times of Malta,https://timesofmalta.com,Watch: Msida flyover to open by end of year as...,'The flyover will allow us to create a new ope...,Daniel Ellul,2025-10-12,The Msida flyover will open by the end of the ...,https://cdn-attachments.timesofmalta.com/f47de...,...,2025-10-12 16:57:00.930159+00,"{Msida,National,Traffic}",{},0,NaN,2025-10-12,2025-10-12 00:00:00,Medium,1.0,0.0
317,496586,https://timesofmalta.com/article/today-front-p...,Times of Malta,https://timesofmalta.com,Today's front pages,The top stories in Malta's newspapers,Times of Malta,2025-10-13,The following are the top stories in Malta's n...,https://cdn-attachments.timesofmalta.com/28065...,...,2025-10-13 08:04:55.910209+00,"{Media,National,""Social Media"",Traffic}",{},0,NaN,2025-10-13,2025-10-13 00:00:00,Medium,0.0,0.0
318,496577,https://timesofmalta.com/article/traffic-overt...,Times of Malta,https://timesofmalta.com,Traffic overtakes cost of living to become peo...,Poll data suggests frustration on Maltese road...,Bertrand Borg,2025-10-13,"Traffic, parking and public transport-related ...",https://cdn-attachments.timesofmalta.com/861ca...,...,2025-10-13 05:03:22.837075+00,"{National,Politics,Traffic}",{},0,NaN,2025-10-13,2025-10-13 00:00:00,Low,0.0,0.0
319,496733,https://timesofmalta.com/article/employer-clea...,Times of Malta,https://ti

### Police Press Releases

#### Methods

In [12]:
def extract_press_release_accident_datetime(content: str, published_date: str) -> dict:
    """
    Extract accident date and time from police press release content.
    
    Returns:
        Dictionary with accident_datetime, accident_date, accident_time, time_confidence
    """
    result = {
        'accident_datetime': None,
        'accident_date': None,
        'accident_time': None,
        'time_confidence': 'low'
    }
    
    if pd.isna(content) or pd.isna(published_date):
        return result
    
    content_lower = content.lower()
    published_dt = pd.to_datetime(published_date)
    extracted_time = None
    
    # HIGH confidence: Exact time with "hrs"
    hrs_patterns = [
        (r'at around (\d{4})\s*hrs', 1), (r'at (\d{4})\s*hrs', 1),
        (r'around (\d{4})\s*hrs', 1), (r'\((\d{4})\s*hrs\)', 1),
        (r'\w+\s*\((\d{4})\s*hrs\)', 1),
        (r'at around (\d{1,2}):(\d{2})\s*hrs', 2), (r'at (\d{1,2}):(\d{2})\s*hrs', 2),
        (r'around (\d{1,2}):(\d{2})\s*hrs', 2),
        (r'at around (\d{1,2})\.(\d{2})\s*hrs', 2), (r'at (\d{1,2})\.(\d{2})\s*hrs', 2),
    ]
    
    for pattern, num_groups in hrs_patterns:
        match = re.search(pattern, content_lower)
        if match:
            if num_groups == 1:
                time_str = match.group(1)
                if len(time_str) == 4:
                    hour, minute = int(time_str[:2]), int(time_str[2:])
                    if 0 <= hour <= 23 and 0 <= minute <= 59:
                        extracted_time = f"{hour:02d}:{minute:02d}"
                        result['time_confidence'] = 'high'
                        break
            else:
                hour, minute = int(match.group(1)), int(match.group(2))
                if 0 <= hour <= 23 and 0 <= minute <= 59:
                    extracted_time = f"{hour:02d}:{minute:02d}"
                    result['time_confidence'] = 'high'
                    break
    
    # HIGH confidence: Standard time formats without "hrs"
    if not extracted_time:
        time_patterns = [
            (r'at around (\d{1,2}):(\d{2})', 2), (r'at (\d{1,2}):(\d{2})', 2),
            (r'around (\d{1,2}):(\d{2})', 2),
            (r'at around (\d{1,2})\.(\d{2})', 2), (r'at (\d{1,2})\.(\d{2})', 2),
        ]
        for pattern, _ in time_patterns:
            match = re.search(pattern, content_lower)
            if match:
                hour, minute = int(match.group(1)), int(match.group(2))
                if 0 <= hour <= 23 and 0 <= minute <= 59:
                    extracted_time = f"{hour:02d}:{minute:02d}"
                    result['time_confidence'] = 'high'
                    break
    
    # MEDIUM-HIGH confidence: Specific time markers
    if not extracted_time:
        time_markers = [
            (r'\bmidnight\b', '00:00', 'high'), (r'\bnoon\b|\bmidday\b', '12:00', 'high'),
            (r'\bdawn\b|\bsunrise\b', '06:00', 'medium'), (r'\bdusk\b|\bsunset\b', '19:00', 'medium'),
        ]
        for pattern, time, conf in time_markers:
            if re.search(pattern, content_lower):
                extracted_time, result['time_confidence'] = time, conf
                break
    
    # MEDIUM confidence: Time ranges (calculate midpoint)
    if not extracted_time:
        match = re.search(r'between (\d{1,2})[:\.]?(\d{2})?\s*(?:and|&|-)\s*(\d{1,2})[:\.]?(\d{2})?', content_lower)
        if match:
            hour1 = int(match.group(1))
            min1 = int(match.group(2)) if match.group(2) else 0
            hour2 = int(match.group(3))
            min2 = int(match.group(4)) if match.group(4) else 0
            if 0 <= hour1 <= 23 and 0 <= hour2 <= 23:
                mid_minutes = ((hour1 * 60 + min1) + (hour2 * 60 + min2)) // 2
                extracted_time = f"{mid_minutes // 60:02d}:{mid_minutes % 60:02d}"
                result['time_confidence'] = 'medium'
    
    # MEDIUM confidence: General time periods
    if not extracted_time:
        time_periods = [
            (r'\bearly hours\b', '03:00'), (r'\blate hours\b|\blate at night\b', '23:00'),
            (r'\bearly morning\b', '06:00'), (r'\bmorning\b', '09:00'),
            (r'\bafternoon\b', '15:00'), (r'\bevening\b', '19:00'), (r'\bnight\b', '22:00'),
        ]
        for pattern, time in time_periods:
            if re.search(pattern, content_lower):
                extracted_time, result['time_confidence'] = time, 'medium'
                break
    
    # Date extraction
    accident_date = None
    explicit_patterns = [
        (r'on (\d{1,2}(?:st|nd|rd|th)?\s+(?:january|february|march|april|may|june|july|august|september|october|november|december))', '%d %B'),
        (r'on ((?:january|february|march|april|may|june|july|august|september|october|november|december)\s+\d{1,2}(?:st|nd|rd|th)?)', '%B %d'),
    ]
    
    for pattern, date_format in explicit_patterns:
        match = re.search(pattern, content_lower)
        if match:
            try:
                date_str = re.sub(r'(st|nd|rd|th)', '', match.group(1))
                accident_date = pd.to_datetime(date_str, format=date_format).replace(year=published_dt.year)
                if accident_date > published_dt + timedelta(days=30):
                    accident_date = accident_date.replace(year=published_dt.year - 1)
                break
            except:
                pass
    
    if accident_date is None:
        if re.search(r'^\s*today[,\s]|^today at', content_lower):
            accident_date = published_dt
        elif re.search(r'^\s*yesterday[,\s]|^yesterday at', content_lower):
            accident_date = published_dt - timedelta(days=1)
        elif re.search(r'\bthis morning\b', content_lower):
            accident_date = published_dt
        elif re.search(r'\blast night\b', content_lower):
            accident_date = published_dt - timedelta(days=1)
        elif re.search(r'\blast evening\b', content_lower):
            accident_date = published_dt - timedelta(days=1)
        elif match := re.search(r'\blast (monday|tuesday|wednesday|thursday|friday|saturday|sunday)', content_lower):
            days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
            days_back = (published_dt.weekday() - days.index(match.group(1))) % 7 or 7
            accident_date = published_dt - timedelta(days=days_back)
        else:
            accident_date = published_dt
    
    # Combine date and time
    if accident_date is not None:
        result['accident_date'] = accident_date.date()
        if extracted_time:
            try:
                hour, minute = map(int, extracted_time.split(':'))
                result['accident_datetime'] = accident_date.replace(hour=hour, minute=minute, second=0)
                result['accident_time'] = extracted_time
            except:
                result['accident_datetime'] = accident_date
        else:
            result['accident_datetime'] = accident_date
    
    return result

In [13]:
def apply_press_releases_manual_corrections(df, corrections):
    """Apply manual time/date corrections to the dataframe."""
    df = df.copy()
    corrections_applied = 0
    
    for release_id, correction_data in corrections.items():
        mask = df['release_id'] == release_id
        if not mask.any():
            print(f"Release ID {release_id} not found")
            continue
        
        idx = df[mask].index[0]
        
        # Determine date
        if 'date' in correction_data:
            new_date = pd.to_datetime(correction_data['date'])
        else:
            new_date = pd.to_datetime(df.loc[idx, 'accident_date'] if pd.notna(df.loc[idx, 'accident_date']) 
                                     else df.loc[idx, 'date_published'])
        
        # Apply time
        if 'time' in correction_data:
            try:
                hour, minute = map(int, correction_data['time'].split(':'))
                if 0 <= hour <= 23 and 0 <= minute <= 59:
                    df.loc[idx, 'accident_time'] = f"{hour:02d}:{minute:02d}"
                    df.loc[idx, 'accident_datetime'] = new_date.replace(hour=hour, minute=minute, second=0)
                    df.loc[idx, 'accident_date'] = new_date.date()
                    df.loc[idx, 'time_confidence'] = 'manual'
                    delay_hours = (pd.to_datetime(df.loc[idx, 'date_published']) - df.loc[idx, 'accident_datetime']).total_seconds() / 3600
                    df.loc[idx, 'publication_delay_hours'] = max(0, delay_hours)
                    corrections_applied += 1
                    print(f"Release ID {release_id}: {correction_data['time']}")
                else:
                    print(f"Invalid time for {release_id}: {correction_data['time']}")
            except Exception as e:
                print(f"Error for {release_id}: {e}")
    
    return df, corrections_applied

#### Main Logic

In [14]:
regex_extract_press_releases_csv = f"{regex_extract_data_folder}/press_releases.csv"

if os.path.isfile(regex_extract_press_releases_csv):
    print("Regex Extraction from police press releases was already done")
else:
    regex_extract_press_releases_df = police_press_releases_df.copy()
    
    extraction_results = regex_extract_press_releases_df.apply(
        lambda row: extract_press_release_accident_datetime(row['content'], row['date_published']), axis=1
    )
    
    # add columns
    extraction_df = pd.DataFrame(extraction_results.tolist())
    for col in ['accident_datetime', 'accident_date', 'accident_time', 'time_confidence']:
        regex_extract_press_releases_df[col] = extraction_df[col]
    
    # add derived features
    regex_extract_press_releases_df['accident_is_weekend'] = pd.to_datetime(regex_extract_press_releases_df['accident_datetime']).dt.dayofweek.isin([5, 6]).astype(int)
    
    regex_extract_press_releases_df['publication_delay_hours'] = ((
        pd.to_datetime(regex_extract_press_releases_df['date_published']) - 
        pd.to_datetime(regex_extract_press_releases_df['accident_datetime'])
    ).dt.total_seconds() / 3600).apply(lambda x: max(0, x) if pd.notna(x) else x)

    # Display low confidence records
    low_conf = regex_extract_press_releases_df[regex_extract_press_releases_df['time_confidence'] == 'low']
    print(f"Low confidence records: {len(low_conf)}")
    
    if len(low_conf) > 0:
        for idx, row in low_conf.head(10).iterrows():
            print(f"ID: {row['release_id']} | Published: {row['date_published']} | Current: {row['accident_time']}")
            print(f"Content: {row['content'][:150]}...")
            print("-" * 80)

    # manual corrections dictionary - add your corrections here
    manual_corrections = {
        25: {'time': '07:45'},
        76: {'time': '17:45'},
        77: {'time': '10:15'}
        # Add more: release_id: {'time': 'HH:MM', 'date': 'YYYY-MM-DD'}
    }

    if manual_corrections:
        regex_extract_press_releases_df, num_applied = apply_press_releases_manual_corrections(regex_extract_press_releases_df, manual_corrections)
        print(f"\n{num_applied} manual corrections applied")
    else:
        print("No manual corrections defined")

    # show stats
    print(f"High: {(regex_extract_press_releases_df['time_confidence'] == 'high').sum()} | Medium: {(regex_extract_press_releases_df['time_confidence'] == 'medium').sum()} | Low: {(regex_extract_press_releases_df['time_confidence'] == 'low').sum()}")
    print(f"\nTotal press releases processed: {len(regex_extract_press_releases_df)}")

    columns_to_save = [col for col in regex_extract_press_releases_df.columns if col not in ['accident_day', 'accident_hour']]
    regex_extract_press_releases_df[columns_to_save].to_csv(regex_extract_press_releases_csv, index=False)

    print(f"✓ Saved {len(regex_extract_press_releases_df)} articles to {regex_extract_press_releases_csv}")
    print(f"  - Articles with datetime extracted: {regex_extract_press_releases_df['accident_datetime'].notna().sum()}")

regex_extract_press_releases_df = pd.read_csv(regex_extract_press_releases_csv)
display(regex_extract_press_releases_df)

Low confidence records: 4
ID: 16 | Published: 2025-04-17 | Current: None
Content: An 80-year-old man, a resident of Santa Luċija, was taken to Mater Dei Hospital for some injuries he sustained in a traffic accident in Luqa Road, Luq...
--------------------------------------------------------------------------------
ID: 25 | Published: 2024-12-27 | Current: None
Content: Today, at about 0745hrs, the Police were informed of a traffic accident in Triq Diċembru Tlettax, Marsa.The Police went immediately on site and prelim...
--------------------------------------------------------------------------------
ID: 76 | Published: 2025-04-14 | Current: None
Content: Yesterday, at about 1745hrs, the Police were informed of a traffic accident in Birżebbuġa Road, Birżebbuġa.Preliminary investigations found that a col...
--------------------------------------------------------------------------------
ID: 77 | Published: 2025-07-24 | Current: None
Content: A 65-year-old man, a resident of Mellieħa, wa

,release_id,title,date_published,date_modified,content,accident_datetime,accident_date,accident_time,time_confidence,accident_is_weekend,publication_delay_hours
0,1,Collision between a car and a motorbike in Żur...,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were info...",2025-10-09 09:30:00,2025-10-09,09:30,high,0,0.00
1,2,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were ...",2025-06-19 18:30:00,2025-06-19,18:30,high,0,5.50
2,3,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were info...",2025-05-12 08:00:00,2025-05-12,08:00,high,0,0.00
3,4,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were ...",2025-07-29 18:00:00,2025-07-29,18:00,high,0,6.00
4,5,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the ev...",2025-04-06 20:45:00,2025-04-06,20:45,high,1,3.25
...,...,...,...,...,...,...,...,...,...,...,...
106,107,Motorcycle accident in Attard,2025-02-05,2025-02-05,"A 52-year-old man and residing in Ħaż-Żebbuġ, ...",2025-02-05 09:00:00,2025-02-05,09:00,high,0,0.00
107,108,Naxxar traffic accident,2024-12-19,2024-12-19,"Today, at around 1045hrs, the Police were info...",2024-12-19 10:45:00,2024-12-19,10:45,high,0,0.00
108,109,Żebbuġ traffic accident,2025-03-16,2025-03-16,"Today, at around 0800hrs, the Police were info...",2025-03-16 08:00:00,2025-03-16,08:00,high,1,0.00
109,110,Collision between a car and e-scooter,2025-07-18,2025-07-18,"Yesterday, at around 2215 hrs, the Police were...",2025-07-17 22:15:00,2025-07-17,22:15,high,0,1.75


## 2. LLM Extraction

Extract features from LLM.

### Variables

In [15]:
llm_extract_data_folder = f"{staging_data_folder}/llm_extract"

### Methods

In [16]:
def extract_llm_output(json_str: str) -> dict:
    return json.loads(json_str.replace("```json", "").replace("```", ""))

In [17]:
def extract_features_from_df(
    pd_df: pd.DataFrame,
    id_column: str,
    prompt: str,
    json_save_path: str
) -> None:
    client = OpenAI(api_key=OPENAI_API_KEY)
    results = []
    
    for index, row in pd_df.iterrows():
        id_value = row[id_column]
        input_text = row["llm_input_text"]

        retry_count = 0
        success = False
    
        print(f"Processing row with {id_column} '{id_value}'...")   

        while retry_count < 3 and not success:
            try:
                response = client.responses.create(
                    model="o4-mini-2025-04-16",
                    instructions=prompt,
                    input=input_text,
                )
                
                llm_output = extract_llm_output(response.output_text)
                llm_output["id_column"] = id_value
                llm_output["input_text"] = input_text
                results.append(llm_output)

                print(f"Successfully processed row with {id_column} '{id_value}'")
                success = True
            except Exception as e:
                retry_count += 1
                print(f"Error for row with {id_column} '{id_value}' (attempt {retry_count}/3): {e}")

                if retry_count < 3:
                    time.sleep(2)  # backoff retry delay
                else:
                    print(f"Failed to process row with {id_column} '{id_value}' after 3 attempts")
        
    with open(json_save_path, 'w') as f:
        json.dump(results, f)

### News Articles

Extract features from news articles using LLM.

#### Prompt

In [18]:
NEWS_ARTICLES_PROMPT = """
You are a helpful data entry assistant whose responsibility is extracting traffic accident data from news articles.
The following is such a news article. Please extract details of the accident and return them in a JSON dict with keys:

- 'is_accident' (boolean) — true if the news article describes an actual traffic accident, false otherwise.
- If 'is_accident' is true, include the following additional keys:
    -'accident_datetime'
    -'street'
    -'city'
    -'number_injured'
    -'accident_severity'
    -'drivers' (a list of objects, each with the following keys:)
        -'vehicle_type'
        -'vehicle_damage_severity'
        -'driver_age'
        -'driver_gender'
        -'is_victim' (boolean)

Please ensure that:
-'incident_datetime' is in the format 'YYYY-MM-DD HH:MM' (24-hour format) if possible.
-'number_injured' is an integer greater or equal to 0
-'accident_severity' which relates to how severe the accident in terms of human injuries and and is one of: 'No Injuries', 'Minor', 'Serious' or 'Fatal'
-'driver_gender' is either 'M' or 'F'.
-'vehicle_damage_severity' is one of: 'No damage', 'Minor' or 'Major' where 'Minor' means small damages and 'Major' means total loss or big damages

Please only return JSON—do not add any other text! If values are missing, set them to the string: "none".
"""

#### Main Logic

In [19]:
llm_extract_news_articles_csv = f"{llm_extract_data_folder}/news_articles.csv"

if os.path.isfile(llm_extract_news_articles_csv):
    print("LLM Extraction from news articles was already done")
else:
    llm_input_articles_df = local_news_articles_df[
        [
            "article_id", # article id to trace back
            "title",
            "subtitle",
            "content",
            "publish_date",
        ]
    ]
    
    llm_input_articles_df["llm_input_text"] = (
        "Title: " + local_news_articles_df["title"].fillna("") + "\n" +
        "Subtitle: " + local_news_articles_df["subtitle"].fillna("") + "\n" +
        "Content: " + local_news_articles_df["content"].fillna("") + "\n" +
        "Publish Date: " + local_news_articles_df["publish_date"].astype(str).fillna("none")
    )

    raw_articles_json = f"{llm_extract_data_folder}/raw_articles.json"

    if os.path.isfile(raw_articles_json):
        print("LLM Feature Extraction from news articles was already done and saved to JSON")
    else:
        extract_features_from_df(
            pd_df=llm_input_articles_df,
            id_column="article_id",
            prompt=NEWS_ARTICLES_PROMPT,
            json_save_path=raw_articles_json,
        )

    llm_extract_news_articles_df = pd.read_json(raw_articles_json)
    llm_extract_news_articles_df.to_csv(llm_extract_news_articles_csv)

    print(f"✓ Saved {len(llm_extract_news_articles_df)} articles to {llm_extract_news_articles_csv}")

llm_extract_news_articles_df = pd.read_csv(llm_extract_news_articles_csv)
display(llm_extract_news_articles_df)

LLM Feature Extraction from news articles was already done and saved to JSON
✓ Saved 318 articles to ../../data/staging/llm_extract/news_articles.csv


/tmp/ipykernel_15994/2644914794.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  llm_input_articles_df["llm_input_text"] = (


,Unnamed: 0,is_accident,accident_datetime,street,city,number_injured,accident_severity,drivers,id_column,input_text,accidents
0,0,True,2024-12-04 17:00,Regional Road,St Julian's,0,No Injuries,"[{'vehicle_type': 'Toyota Yaris', 'vehicle_dam...",4208,Title: Driver stuck in traffic says speeding L...,NaN
1,1,False,NaN,NaN,NaN,NaN,NaN,NaN,4167,Title: PN slams government for diverting EU bu...,NaN
2,2,True,2024-12-11 17:00,St Paul's Bay bypass,St Paul's Bay,1,Serious,"[{'vehicle_type': 'Motorcycle', 'vehicle_damag...",4093,Title: Motorcyclist seriously hurt in St Paul'...,NaN
3,3,True,2024-12-11 17:00,St Paul’s Bay bypass,St Paul's Bay,1,Serious,"[{'vehicle_type': 'Motorcycle', 'vehicle_damag...",4110,Title: Skip involved in horror St Paul’s Bay b...,NaN
4,4,True,NaN,NaN,NaN,NaN,NaN,NaN,4066,"Title: Two people, including teenage girl, cri...","[{'accident_datetime': '2024-12-14 17:30', 'st..."
...,...,...,...,...,...,...,...,...,...,...,...
313,313,False,NaN,NaN,NaN,NaN,NaN,NaN,496574,Title: Watch: Msida flyover to open by end of ...,NaN
314,314,False,NaN,NaN,NaN,NaN,NaN,NaN,496586,Title: Today's front pages\nSubtitle: The top ...,NaN
315,315,False,NaN,NaN,NaN,NaN,NaN,NaN,496577,Title: Traffic overtakes cost of living to bec...,NaN
316,316,False,NaN,NaN,NaN,NaN,NaN,NaN,496733,Title: Employer cleared of responsibility for ...,NaN


### Police Press Releases

#### Prompt

In [20]:
POLICE_PRESS_RELEASES_PROMPT = """
You are a helpful data entry assistant whose responsibility is extracting traffic accident data from police press releases.
The following is such a press release. Please extract details of the accident and return them in a JSON dict with keys:

- 'is_accident' (boolean) — true if the news article describes an actual traffic accident, false otherwise.
- If 'is_accident' is true, include the following additional keys:
    -'accident_datetime'
    -'street'
    -'city'
    -'number_injured'
    -'accident_severity'
    -'drivers' (a list of objects, each with the following keys:)
        -'vehicle_type'
        -'vehicle_damage_severity'
        -'driver_age'
        -'driver_gender'
        -'is_victim' (boolean)

Please ensure that:
-'incident_datetime' is in the format 'YYYY-MM-DD HH:MM' (24-hour format) if possible.
-'number_injured' is an integer greater or equal to 0
-'accident_severity' which relates to how severe the accident in terms of human injuries and and is one of: 'No Injuries', 'Minor', 'Serious' or 'Fatal'
-'driver_gender' is either 'M' or 'F'.
-'vehicle_damage_severity' is one of: 'No damage', 'Minor' or 'Major' where 'Minor' means small damages and 'Major' means total loss or big damages

Please only return JSON—do not add any other text! If values are missing, set them to the string: "none".
"""

#### Main Logic

In [21]:
llm_extract_press_releases_csv = f"{llm_extract_data_folder}/press_releases.csv"

if os.path.isfile(llm_extract_press_releases_csv):
    print("LLM Extraction from police press releases was already done")
else:
    llm_input_press_releases_df = police_press_releases_df[
        [
            "release_id", # release_id
            "title",
            "date_published",
            "content",
        ]
    ]
    
    llm_input_press_releases_df["llm_input_text"] = (
        "Title: " + police_press_releases_df["title"].fillna("") + "\n" +
        "Content: " + police_press_releases_df["content"].fillna("") + "\n" +
        "Publish Date: " + police_press_releases_df["date_published"].astype(str).fillna("none")
    )
    
    raw_press_releases_json = f"{llm_extract_data_folder}/raw_press_releases.json"
    
    if os.path.isfile(raw_press_releases_json):
        print("LLM Feature Extraction from police press releases was already done and saved to JSON")
    else:
        extract_features_from_df(
            pd_df=llm_input_press_releases_df,
            id_column="release_id",
            prompt=POLICE_PRESS_RELEASES_PROMPT,
            json_save_path=raw_press_releases_json,
        )

    llm_extract_press_releases_df = pd.read_json(raw_press_releases_json)
    llm_extract_press_releases_df.to_csv(llm_extract_press_releases_csv)

    print(f"✓ Saved {len(llm_extract_press_releases_df)} articles to {llm_extract_press_releases_csv}")

llm_extract_press_releases_df = pd.read_csv(llm_extract_press_releases_csv)
display(llm_extract_press_releases_df)

LLM Feature Extraction from police press releases was already done and saved to JSON
✓ Saved 111 articles to ../../data/staging/llm_extract/press_releases.csv


,Unnamed: 0,is_accident,accident_datetime,street,city,number_injured,accident_severity,drivers,id_column,input_text,accidents
0,0,True,2025-10-09 09:30,Triq il-Belt Valletta,Żurrieq,1.0,Serious,"[{'vehicle_type': 'Car', 'vehicle_damage_sever...",1,Title: Collision between a car and a motorbike...,NaN
1,1,True,2025-06-19 18:30,Triq Dawret il-Gudja,Gudja,1.0,Serious,"[{'vehicle_type': 'Honda fit', 'vehicle_damage...",2,Title: Car-motorcycle traffic accident\nConten...,NaN
2,2,True,2025-05-12 08:00,Valley Road,Qormi,1.0,Serious,"[{'vehicle_type': 'Ford Transit', 'vehicle_dam...",3,Title: Car-motorcycle collision in Ħal Qormi\n...,NaN
3,3,True,2025-07-29 18:00,Triq Dawret Ħal Għaxaq,Għaxaq,1.0,Serious,"[{'vehicle_type': 'Volvo XC60', 'vehicle_damag...",4,Title: Collision between motorcycle and car in...,NaN
4,4,True,2025-04-06 20:45,Triq il-Buqana,Rabat,1.0,Serious,"[{'vehicle_type': 'Car', 'vehicle_damage_sever...",5,Title: Car-motorcycle collision\nContent: Yest...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
106,106,True,2025-02-05 09:00,Vjal L-Istadium Nazzjonali,Attard,1.0,Serious,"[{'vehicle_type': 'Motorcycle', 'vehicle_damag...",107,Title: Motorcycle accident in Attard\nContent:...,NaN
107,107,True,2024-12-19 10:45,Triq il-Ġermanja,Naxxar,1.0,Serious,"[{'vehicle_type': 'Toyota Vitz', 'vehicle_dama...",108,Title: Naxxar traffic accident\nContent: Today...,NaN
108,108,True,2025-03-16 08:00,Vjal il-Helsien,Zebbug,2.0,Serious,"[{'vehicle_type': 'Peugeot 306', 'vehicle_dama...",109,Title: Żebbuġ traffic accident \nContent: Tod...,NaN
109,109,True,2025-07-17 22:15,Triq il-Wied ta’ Birkirkara,Birkirkara,1.0,Serious,"[{'vehicle_type': 'Car', 'vehicle_damage_sever...",110,Title: Collision between a car and e-scooter\n...,NaN


## 3. Join Extraction Together

Join the regex extracted data and the LLM extracted data together for manual auditing.

- Datetime feature extraction using Regex.
- General feature extraction using LLM.

### Variables

In [22]:
joined_extract_data_folder = f"{staging_data_folder}/joined_extract"
og_prefix = "og_"
regex_dtime_prefix = "regxdt_"
llm_prefix = "llm_"

### Methods

In [23]:
def parse_llm_drivers(x):
    if pd.isna(x) or x.strip() == "":
        return []
    return ast.literal_eval(x)

### News Articles

In [24]:
joined_news_articles_csv = f"{joined_extract_data_folder}/news_articles.csv"

if os.path.isfile(joined_news_articles_csv):
    print("Extraction CSVs were already joined together")
else:
    to_join_news_articles_df = (
        local_news_articles_df[[
            "article_id",
            "url",
            "source_name",
            "source_url",
            "title",
            "subtitle",
            # "author_name", -> not interested in the name of the author
            "publish_date",
            "content",
            "top_image_url",
            "top_image_caption",
            "created_at",
            "tags",
            # "categories" -> always empty set, not interested in this column
        ]]
        .rename(columns={
            "article_id": "article_id",
            "url": f"{og_prefix}url",
            "source_name": f"{og_prefix}source_name",
            "source_url": f"{og_prefix}source_url",
            "title": f"{og_prefix}title",
            "subtitle": f"{og_prefix}subtitle",
            "publish_date": f"{og_prefix}publish_date",
            "content": f"{og_prefix}content",
            "top_image_url": f"{og_prefix}top_image_url",
            "top_image_caption": f"{og_prefix}top_image_caption",
            "created_at": f"{og_prefix}created_at",
            "tags": f"{og_prefix}tags",
        })
    )
    
    to_join_regex_news_articles_df = (
        regex_extract_news_articles_df[[
            "article_id",
            "accident_datetime",
        ]]
        .rename(columns={
            "article_id": "article_id",
            "accident_datetime": f"{regex_dtime_prefix}accident_datetime",
        })
    )

    to_join_llm_news_articles_df = (
        llm_extract_news_articles_df[[
            "id_column", # article_id
            "is_accident",
            "street",
            "city",
            "number_injured",
            "accident_severity",
            "drivers",
        ]]
        .rename(columns={
            "id_column": "article_id",
            "is_accident": f"{llm_prefix}is_accident",
            "street": f"{llm_prefix}street",
            "city": f"{llm_prefix}city",
            "number_injured": f"{llm_prefix}number_injured",
            "accident_severity": f"{llm_prefix}accident_severity",
            "drivers": f"{llm_prefix}drivers",
        })
    )

    joined_news_articles_df = (
        to_join_news_articles_df
        .merge(to_join_regex_news_articles_df, on="article_id", how="left")
        .merge(to_join_llm_news_articles_df, on="article_id", how="left")
    )
    
    joined_news_articles_df["llm_drivers"] = joined_news_articles_df["llm_drivers"].apply(parse_llm_drivers)
    exploded_news_articles_df = joined_news_articles_df.explode("llm_drivers", ignore_index=True)
    
    joined_news_articles_df = pd.concat(
        [
            exploded_news_articles_df.drop(columns=["llm_drivers"]),
            pd.json_normalize(exploded_news_articles_df["llm_drivers"])
        ],
        axis=1
    ).rename(columns={
        "vehicle_type": f"{llm_prefix}_vehicle_type",
        "vehicle_damage_severity": f"{llm_prefix}vehicle_damage_severity",
        "driver_age": f"{llm_prefix}driver_age",
        "driver_gender": f"{llm_prefix}driver_gender",
        "is_victim": f"{llm_prefix}is_victim",
    })
    
    joined_news_articles_df.to_csv(joined_news_articles_csv)

    print(f"✓ Saved {len(joined_news_articles_df)} articles to {joined_news_articles_csv}")


joined_news_articles_df = pd.read_csv(joined_news_articles_csv)
display(joined_news_articles_df)

✓ Saved 453 articles to ../../data/staging/joined_extract/news_articles.csv


,Unnamed: 0,article_id,og_url,og_source_name,og_source_url,og_title,og_subtitle,og_publish_date,og_content,og_top_image_url,...,llm_is_accident,llm_street,llm_city,llm_number_injured,llm_accident_severity,llm__vehicle_type,llm_vehicle_damage_severity,llm_driver_age,llm_driver_gender,llm_is_victim
0,0,4208,https://timesofmalta.com/article/driver-stuck-...,Times of Malta,https://timesofmalta.com,Driver stuck in traffic says speeding LESA car...,‘I was shocked at that moment but more so frus...,2024-12-07,A motorist claims his car mirror was shattered...,https://cdn-attachments.timesofmalta.com/706da...,...,True,Regional Road,St Julian's,0,No Injuries,Toyota Yaris,Minor,78,M,True
1,1,4208,https://timesofmalta.com/article/driver-stuck-...,Times of Malta,https://timesofmalta.com,Driver stuck in traffic says speeding LESA car...,‘I was shocked at that moment but more so frus...,2024-12-07,A motorist claims his car mirror was shattered...,https://cdn-attachments.timesofmalta.com/706da...,...,True,Regional Road,St Julian's,0,No Injuries,LESA vehicle,No damage,none,none,False
2,2,4167,https://timesofmalta.com/article/pn-slams-gove...,Times of Malta,https://timesofmalta.com,PN slams government for diverting EU bus funds...,"'By encouraging the use of private cars, the g...",2024-12-09,The PN on Monday slammed the government for di...,https://cdn-attachments.timesofmalta.com/d9afe...,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4093,https://timesofmalta.com/article/motorcyclist-...,Times of Malta,https://timesofmalta.com,Motorcyclist seriously hurt in St Paul's Bay b...,Residents complained several times about inade...,2024-12-11,A motorcyclist was rushed to hospital in a cri...,https://cdn-attachments.timesofmalta.com/633f6...,...,True,St Paul's Bay bypass,St Paul's Bay,1,Serious,Motorcycle,Major,54,M,True
4,4,4110,https://timesofmalta.com/article/skip-involved...,Times of Malta,https://timesofmalta.com,Skip involved in horror St Paul’s Bay bypass c...,Motorcyclist hurt in crash on Wednesday evenin...,2024-12-12,A private contractor who placed a skip on St P...,https://cdn-attachments.timesofmalta.com/fc23e...,...,True,St Paul’s Bay bypass,St Paul's Bay,1,Serious,Motorcycle,none,54,M,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,448,496574,https://timesofmalta.com/article/watch-msida-f...,Times of Malta,https://timesofmalta.com,Watch: Msida flyover to open by end of year as...,'The flyover will allow us to create a new ope...,2025-10-12,The Msida flyover will open by the end of the ...,https://cdn-attachments.timesofmalta.com/f47de...,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,449,496586,https://timesofmalta.com/article/today-front-p...,Times of Malta,https://timesofmalta.com,Today's front pages,The top stories in Malta's newspapers,2025-10-13,The following are the top stories in Malta's n...,https://cdn-attachments.timesofmalta.com/28065...,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,450,496577,https://timesofmalta.com/article/traffic-overt...,Times of Malta,https://timesofmalta.com,Traffic overtakes cost of living to become peo...,Poll data suggests frustration on Maltese road...,2025-10-13,"Traffic, parking and public transport-related ...",https://cdn-attachments.timesofmalta.com/861ca...,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
451,451,496733,https://timesofmalta.com/article/employer-clea...,Times of Malta,https://timesofmalta.com,Employer cleared of responsibility for young w...,"Court raps police, OHSA for not working togeth...",2025-10-14,A court has sharply criticised the police and ...,https://cdn-attachments.timesofmalta.com/2d4fb...,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Police Press Releases

In [25]:
joined_press_releases_csv = f"{joined_extract_data_folder}/press_releases.csv"

if os.path.isfile(joined_press_releases_csv):
    print("Extraction CSVs were already joined together")
else:
    to_join_press_releases_df = (
        police_press_releases_df[[
            "release_id",
            "title",
            "content",
            "date_published",
            "date_modified",
        ]]
        .rename(columns={
            "release_id": "release_id",
            "title": f"{og_prefix}title",
            "content": f"{og_prefix}content",
            "date_published": f"{og_prefix}date_published",
            "date_modified": f"{og_prefix}date_modified",
        })
    )
    
    to_join_regex_press_releases_df = (
        regex_extract_press_releases_df[[
            "release_id",
            "accident_datetime",
        ]]
        .rename(columns={
            "release_id": "release_id",
            "accident_datetime": f"{regex_dtime_prefix}accident_datetime",
        })
    )

    to_join_llm_press_releases_df = (
        llm_extract_press_releases_df[[
            "id_column", # release_id
            "is_accident",
            "street",
            "city",
            "number_injured",
            "accident_severity",
            "drivers",
        ]]
        .rename(columns={
            "id_column": "release_id",
            "is_accident": f"{llm_prefix}is_accident",
            "street": f"{llm_prefix}street",
            "city": f"{llm_prefix}city",
            "number_injured": f"{llm_prefix}number_injured",
            "accident_severity": f"{llm_prefix}accident_severity",
            "drivers": f"{llm_prefix}drivers",
        })
    )

    joined_press_releases_df = (
        to_join_press_releases_df
        .merge(to_join_regex_press_releases_df, on="release_id", how="left")
        .merge(to_join_llm_press_releases_df, on="release_id", how="left")
    )
    
    joined_press_releases_df["llm_drivers"] = joined_press_releases_df["llm_drivers"].apply(parse_llm_drivers)
    exploded_press_releases_df = joined_press_releases_df.explode("llm_drivers", ignore_index=True)
    
    joined_press_releases_df = pd.concat(
        [
            exploded_press_releases_df.drop(columns=["llm_drivers"]),
            pd.json_normalize(exploded_press_releases_df["llm_drivers"])
        ],
        axis=1
    ).rename(columns={
        "vehicle_type": f"{llm_prefix}_vehicle_type",
        "vehicle_damage_severity": f"{llm_prefix}vehicle_damage_severity",
        "driver_age": f"{llm_prefix}driver_age",
        "driver_gender": f"{llm_prefix}driver_gender",
        "is_victim": f"{llm_prefix}is_victim",
    })
    
    joined_press_releases_df.to_csv(joined_press_releases_csv)

    print(f"✓ Saved {len(joined_press_releases_df)} articles to {joined_press_releases_csv}")

joined_press_releases_df = pd.read_csv(joined_press_releases_csv)
display(joined_press_releases_df)

✓ Saved 175 articles to ../../data/staging/joined_extract/press_releases.csv


,Unnamed: 0,release_id,og_title,og_content,og_date_published,og_date_modified,regxdt_accident_datetime,llm_is_accident,llm_street,llm_city,llm_number_injured,llm_accident_severity,llm__vehicle_type,llm_vehicle_damage_severity,llm_driver_age,llm_driver_gender,llm_is_victim
0,0,1,Collision between a car and a motorbike in Żur...,"Today, at around 0930hrs, the Police were info...",2025-10-09,2025-10-09,2025-10-09 09:30:00,True,Triq il-Belt Valletta,Żurrieq,1.0,Serious,Car,none,67,F,False
1,1,1,Collision between a car and a motorbike in Żur...,"Today, at around 0930hrs, the Police were info...",2025-10-09,2025-10-09,2025-10-09 09:30:00,True,Triq il-Belt Valletta,Żurrieq,1.0,Serious,Motorbike,none,61,M,True
2,2,2,Car-motorcycle traffic accident,"Yesterday, at around 1830hrs, the Police were ...",2025-06-20,2025-06-20,2025-06-19 18:30:00,True,Triq Dawret il-Gudja,Gudja,1.0,Serious,Honda fit,none,64,M,False
3,3,2,Car-motorcycle traffic accident,"Yesterday, at around 1830hrs, the Police were ...",2025-06-20,2025-06-20,2025-06-19 18:30:00,True,Triq Dawret il-Gudja,Gudja,1.0,Serious,Kawasaki Ninja motorcycle,none,23,M,True
4,4,3,Car-motorcycle collision in Ħal Qormi,"Today, at around 0800hrs, the Police were info...",2025-05-12,2025-05-12,2025-05-12 08:00:00,True,Valley Road,Qormi,1.0,Serious,Ford Transit,none,34,M,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,170,109,Żebbuġ traffic accident,"Today, at around 0800hrs, the Police were info...",2025-03-16,2025-03-16,2025-03-16 08:00:00,True,Vjal il-Helsien,Zebbug,2.0,Serious,Peugeot 306,none,59,M,False
171,171,110,Collision between a car and e-scooter,"Yesterday, at around 2215 hrs, the Police were...",2025-07-18,2025-07-18,2025-07-17 22:15:00,True,Triq il-Wied ta’ Birkirkara,Birkirkara,1.0,Serious,Car,none,41,none,False
172,172,110,Collision between a car and e-scooter,"Yesterday, at around 2215 hrs, the Police were...",2025-07-18,2025-07-18,2025-07-17 22:15:00,True,Triq il-Wied ta’ Birkirkara,Birkirkara,1.0,Serious,E-scooter,none,17,none,True
173,173,111,Traffic accident in Gwardamanġa,"Today, at around 0700hrs, the Police were info...",2025-08-12,2025-08-12,2025-08-12 07:00:00,True,St Luke’s Square,Gwardamanġa,2.0,Serious,Volkswagen Caddy,none,62,M,True


## 4. Combine datasets and deduplicate

DataFrames have been manually audited and validated. Some values have been updated.

Now, we combine both of the datasets together and deduplicate. Deduplication will happen programatically to find duplicates and manually to resolve them.

### Variables

In [26]:
audited_data_folder = f"{staging_data_folder}/audited"

In [27]:
audited_news_articles_csv = f"{audited_data_folder}/news_articles.csv"
audited_press_releases_csv = f"{audited_data_folder}/press_releases.csv"

In [28]:
audited_news_articles_df = pd.read_csv(audited_news_articles_csv)

print(f"\nColumn names: {audited_news_articles_df.columns.tolist()}")

display(audited_news_articles_df)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/staging/audited/news_articles.csv'

In [ ]:
audited_press_releases_df = pd.read_csv(audited_press_releases_csv)

print(f"\nColumn names: {audited_press_releases_df.columns.tolist()}")

display(audited_press_releases_df)

In [ ]:
# to-do: combine both of them together, add logic to detect duplicates and remove duplicates manually